# Predicción de Cancelación de Clientes (Churn)

## Contexto
El objetivo de este proyecto es predecir la cancelación de clientes de una empresa de telecomunicaciones a partir de datos históricos de comportamiento, servicios contratados y características del cliente.

## Objetivo
Construir y evaluar modelos de clasificación que permitan identificar clientes con alta probabilidad de cancelar el servicio, utilizando métricas apropiadas para problemas de clasificación desbalanceados.

La evaluación del modelo se realizará utilizando ROC-AUC como métrica principal.

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier

## Carga de datos

In [2]:
contract = pd.read_csv('/datasets/final_provider/contract.csv')
personal = pd.read_csv('/datasets/final_provider/personal.csv')
internet = pd.read_csv('/datasets/final_provider/internet.csv')
phone = pd.read_csv('/datasets/final_provider/phone.csv')

## Limpieza de datos

### Datos de contrato

#### Mostrar información general sobre el dataset de contrato

In [ ]:
contract.head()

In [ ]:
contract.info()

In [ ]:
contract.describe()

In [ ]:
contract.shape

#### Limpiar datos

In [7]:
# creamos una función para convertir nombres de columnas a snake_case
def to_snake_case(col_name):
    # Separar palabras unidas por mayúsculas
    col_name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', col_name)
    # Manejar siglas o mayúsculas seguidas
    col_name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', col_name)
    return col_name.lower()

# creamos una función para limpiar un DataFrame completo
def clean_dataframe(df):
    # Convertir nombres de columnas
    df = df.rename(columns=lambda x : to_snake_case(x))
    # Limpiar espacios no deseados en las columnas tipo object
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip()
    return df

In [ ]:
# Limpiamos el dataframe
contract = clean_dataframe(contract)

# corregimos los tipos de datos
    # begin_date y end_date deben ser datetime
contract['begin_date'] = pd.to_datetime(contract['begin_date'], errors='coerce')
contract['end_date'] = pd.to_datetime(contract['end_date'], errors='coerce')
    # TotalCharges debe ser float64
contract['total_charges'] = pd.to_numeric(contract['total_charges'], errors='coerce')
# revisamos valores ausentes
print('Valores ausentes', contract.isna().sum())
# revisamos filas duplicadas
print('Filas duplicadas', contract.duplicated().sum())

#### Verificamos

In [ ]:
contract.head()

In [ ]:
contract.info()

**Observación:** Después de corregir los tipos de datos aparecieron valores ausentes en las columnas 'end_date' y 'total_charges', más adelante analizaremos a profundidad el significado de esto.

### Datos personales

#### Mostrar información general sobre el dataset de datos personales

In [ ]:
personal.head()

In [ ]:
personal.info()

In [ ]:
personal.describe()

In [ ]:
personal.shape

#### Limpiar datos

In [ ]:
# Limpiamos el dataframe
personal = clean_dataframe(personal)

# revisamos valores ausentes
print('Valores ausentes:\n', personal.isna().sum())
# revisamos filas duplicadas
print('\nFilas duplicadas:\n', personal.duplicated().sum())

#### Verificamos

In [ ]:
personal.head()

In [ ]:
personal.info()

### Plan de internet

#### Mostrar información general sobre el dataset del plan de internet

In [ ]:
internet.head()

In [ ]:
internet.info()

In [ ]:
internet.describe()

In [ ]:
internet.shape

#### Limpiar datos

In [ ]:
# Limpiamos el dataframe
internet = clean_dataframe(internet)

# revisamos valores ausentes
print('Valores ausentes:\n', internet.isna().sum())
# revisamos filas duplicadas
print('\nFilas duplicadas:\n', internet.duplicated().sum())

#### Verificamos

In [ ]:
internet.head()

In [ ]:
internet.info()

### Plan telefónico

#### Mostrar información general sobre el dataset del plan telefónico

In [ ]:
phone.head()

In [ ]:
phone.info()

In [ ]:
phone.describe()

In [ ]:
phone.shape

#### Limpiar datos

In [ ]:
# Limpiamos el dataframe
phone = clean_dataframe(phone)

# revisamos valores ausentes
print('Valores ausentes:\n', phone.isna().sum())
# revisamos filas duplicadas
print('\nFilas duplicadas:\n', phone.duplicated().sum())

#### Verificamos

In [ ]:
phone.head()

In [ ]:
phone.info()

## Análisis exploratorio de datos

### Unión de datasets

In [32]:
# Unificamos las cuatro tablas de datos para poder hacer el análisis exploratorio
df_merged = contract.merge(personal, on='customer_id', how='left')
df_merged = df_merged.merge(internet, on='customer_id', how='left')
full_data = df_merged.merge(phone, on='customer_id', how='left')

In [ ]:
# Verificamos que se hayan conservado correctamente todos los clientes después del merge
print(full_data.shape)
print()
print(full_data['customer_id'].nunique())

### Mostrar información general sobre el dataset final

In [ ]:
full_data.head(50)

In [ ]:
full_data.info()

In [ ]:
full_data.isna().sum()

**Observaciones:**

Existen 5174 valores ausentes en 'end_date', lo que significa que 5174 clientes aún tienen su contrato activo.

Existen 1526 filas con valores ausentes en las columnas correspondientes al dataset internet, lo que significa que 1526 clientes no cuentan con el servicio de internet.

Existen 682 filas con valores ausentes en la columna correspondiente al dataset phone, lo que significa que 682 clientes no cuentan con el servicio de telefonía.

Lo anterior también nos indica que existen 2,208 clientes que poseen un solo servicio, y 4,835 clientes con ambos servicios contratados.

In [ ]:
# Analizamos los clientes que tienen valores ausentes en 'total_charges'
full_data[full_data['total_charges'].isna()]

**Observación:**

Las filas con valores ausentes en 'total_charges' se refieren a clientes que acaban de iniciar su contrato, por lo que aún no han acumulado ningún cargo total.

### Tratar valores ausentes

In [38]:
# Reemplazamos con 'No' los valores ausentes en las columnas correspondientes al dataset de internet
internet_cols = ['internet_service', 'online_security', 'online_backup',
                 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']

for col in internet_cols:
    full_data[col] = full_data[col].fillna('No')

In [ ]:
# Verificamos
print(full_data[internet_cols].isna().sum())

In [40]:
# Reemplazamos con 'No' los valores ausentes en la columna correspondiente al dataset de telefonía
full_data['multiple_lines'] = full_data['multiple_lines'].fillna('No')

In [ ]:
# Verificamos
print(full_data['multiple_lines'].isna().sum())

In [42]:
# Reemplazamos con 0 los valores ausentes en la columna 'total_charges'
full_data['total_charges'] = full_data['total_charges'].fillna(0)

In [ ]:
# Verificamos
print(full_data['total_charges'].isna().sum())

In [ ]:
full_data.head(30)

In [ ]:
full_data.isna().sum()

### Enriquecer datos

In [46]:
# Agregar una variable que indique si el cliente canceló su contrato (esta será nuestra nueva variable objetivo)
full_data['churn'] = full_data['end_date'].notna().astype(int)

# Agregar una variable para indicar la antigüedad del cliente
reference_date = full_data['end_date'].max() # Usamos como referencia la última fecha observada en el dataset para evitar data leakage temporal
full_data['end_date_filled'] = full_data['end_date'].fillna(reference_date)
full_data['tenure'] = (
    (full_data['end_date_filled'] - full_data['begin_date']).dt.days / 30
).round()

# Agregar una variable que indique si el cliente cuenta con el servivio de internet
full_data['has_internet'] = (full_data['internet_service'] != 'No').astype(int)

# Agregar una variable que indique si el cliente cuenta con el servicio de telefonía
full_data['has_phone'] = (full_data['multiple_lines'] != 'No').astype(int)

# Agregar una variable que indique si el cliente cuenta con un solo servicio
full_data['one_service'] = (full_data['has_internet'] + full_data['has_phone'] == 1).astype(int)

# Agregar el mes (por nombre y número) de inicio y fin de contrato
full_data['begin_month'] = full_data['begin_date'].dt.month
full_data['begin_month_name'] = full_data['begin_date'].dt.month_name()
full_data['churn_month'] = full_data['end_date'].dt.month
full_data['churn_month_name'] = full_data['end_date'].dt.month_name()

In [ ]:
full_data.head(10)

In [ ]:
full_data.info()

In [ ]:
full_data.isna().sum()

In [ ]:
full_data.describe()

### Análisis general

In [ ]:
# ANÁLISIS GENERAL
# Creamos un histograma para cada variable numérica
full_data.hist(figsize=(12, 8), bins=30)
plt.tight_layout()
plt.show()

# Creamos un boxplot para cada variable numérica (sin target)
for col in full_data.select_dtypes(include='number'):
    plt.figure(figsize=(6,2))
    sns.boxplot(x=full_data[col])
    plt.title(f'Boxplot - {col}')
    plt.show()

# Observamos la distribución de categorías en cada columna categórica
for col in full_data.select_dtypes(include='object'):
    print(f"\nDistribución de {col}:")
    print(full_data[col].value_counts(dropna=False))

**Observaciones:**

Todas las variables numéricas muestran variabilidad suficiente, rangos adecuados y distribuciones coherentes en los histogramas y diagramas de caja. Ninguna presenta valores atípicos imposibles ni concentraciones extremas. Por lo tanto, todas son apropiadas para el análisis específico segmentado por el target.

En cuanto a las variables categóricas, tras revisar las distribuciones con value_counts(), todas las variables presentaron variedad suficiente en sus categorías como para incluirlas en el análisis específico segmentado por el target. La única excepción es customer_id, ya que es un identificador único y no aporta información útil, por lo que se descartará.

In [52]:
# Filtrar columnas relevantes para análisis específico

# Columnas que representan meses pero deben tratarse como categóricas
meses_numericos = ['begin_month', 'churn_month']

# Detectar columnas binarias numéricas (deben tratarse como categóricas)
binarias_numericas = [
    col for col in full_data.select_dtypes(include=['number']).columns
    if full_data[col].nunique() == 2 and col != 'churn'
]

# Numéricas
numericas_relevantes = [
    col for col in full_data.select_dtypes(include=['number']).columns
    if col not in ['churn'] + binarias_numericas + meses_numericos
]

# Categóricas
categoricas_relevantes = [
    col for col in full_data.select_dtypes(include=['object']).columns
    if col not in ['customer_id']
] + binarias_numericas

# Orden natural de meses por nombre
orden_meses_nombre = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

In [ ]:
# Observamos el balanceo de clases en la variable objetivo
print(full_data['churn'].value_counts(normalize=True))

### Análisis específico

In [ ]:
# ANÁLISIS ESPECÍFICO PARA VARIABLES NUMÉRICAS
for col in numericas_relevantes:

    # Creamos un boxplot segmentado por la variable objetivo
    plt.figure(figsize=(6,3))
    sns.boxplot(x='churn', y=col, data=full_data)
    plt.title(f'Boxplot segmentado por el target - {col}')
    plt.show()

    if (full_data[col].nunique() > 10):
        # Creamos un KDE segmentado por la variable objetivo solo para variables continuas
        plt.figure(figsize=(6,3))
        sns.kdeplot(data=full_data, x=col, hue='churn', common_norm=False)
        plt.title(f'KDE segmentado por el target - {col}')
        plt.show()

    # Mostramos el valor medio de la columna según el target
    print(f"\nValor medio de {col} según el target:")
    print(full_data.groupby('churn')[col].mean())

**Observaciones:**

**Relación de los cargos mensuales con la cancelación del servicio:**

Los clientes que cancelaron presentan cargos mensuales considerablemente más altos. El boxplot muestra que no hay valores anormales. El KDE confirma que los clientes activos se concentran en pagos entre 20 y 40 USD, mientras que quienes cancelaron se concentran principalmente entre 60 y 100 USD. Además, la media de pago mensual también es mayor en el grupo que se dio de baja (74 USD) en comparación con el grupo activo (61 USD). Esto sugiere que los clientes con tarifas más elevadas son más propensos a cancelar su contrato.

**Relación de los cargos totales con la cancelación del servicio:**

Los valores atípicos mostrados en el boxplot sugieren que solo una pequeña parte de los clientes que cancelaron su contrato llegaron a pagar más de 6000 USD totales.
El KDE muestra que los clientes activos se concentran principalmente entre los 0 y 2500 USD totales, mientras que los que cancelaron se concentran entre los 0 y 1500 USD. 
Además, la media del pago total es mayor en el grupo que sigue activo (2549 USD) en comparación con el grupo que canceló su contrato (1531 USD).
Esto tiene sentido ya que los clientes con contrato activo siguen acumulando cargos mientras que a los que ya cancelaron se les deja de cobrar.
Con esto podemos concluir que la cancelación del servicio no se ve influenciada por los cargos totales, sino que los cargos totales reflejan la antigüedad.

**Relación de la antigüedad con la cancelación del servicio:**

Los valores atípicos en el boxplot sugieren que solo una pequeña parte de los clientes que cancelaron su servicio llevaba más de 70 meses de antigüedad.
En el KDE los clientes activos muestran una distribución bastante uniforme entre los 0 y 80 meses de antigüedad, lo que indica una gran estabilidad. Por otro lado, los clientes que cancelaron se concentran mayormente entre los 0 y 20 meses.
El valor medio de antigüedad es claramente mayor en los clientes activos (37 meses), en comparación con los clientes que cancelaron (18 meses).
Todo lo anterior sugiere que existe una gran probabilidad de cancelación con clientes relativamente nuevos, mientras que con clientes antiguos la probabilidad es muy escasa.

In [ ]:
# ANÁLISIS ESPECÍFICO PARA VARIABLES CATEGÓRICAS
for col in categoricas_relevantes:
    
    plt.figure(figsize=(6,3))
    # ordenar correctamente si la columna es de nombre de mes
    if col in ['begin_month_name', 'churn_month_name']:
        sns.barplot(x=col, y='churn', data=full_data, order=orden_meses_nombre)
    else:
        sns.barplot(x=col, y='churn', data=full_data)

    plt.xticks(rotation=45)
    plt.title(f'Barplot - tasa de abandono por categoría de {col}')
    plt.show()

    # Mostramos el valor medio de abandono por categoría
    print(f"\nTasa de abandono según {col}:")
    print(full_data.groupby(col)['churn'].mean())

**Observaciones:**

**Relación del tipo de contrato con la cancelación del servicio:**

La tasa de abandono varía de manera significativa según el tipo de contrato.
Los clientes con contratos mensuales presentan la tasa de cancelación más alta (42%), mientras que quienes tienen contratos anuales y bienales muestran tasas mucho menores (11% y 2%, respectivamente). Esto podría deberse a que los contratos mensuales implican menos compromiso y permiten cancelar fácilmente, lo que los hace mucho más propensos a la deserción. En cambio, los contratos anuales y bienales suelen incluir beneficios, descuentos o penalizaciones por cancelación, lo que reduce drásticamente la probabilidad de cancelación.

**Relación del tipo de facturación con la cancelación del servicio:**

La tasa de abandono para clientes con facturación electrónica (33%) es más del doble que la de clientes con facturación en papel (16%). Esto sugiere que quienes usan facturación electrónica tienden a cancelar con mayor frecuencia. Una explicación podría ser que este grupo de clientes suele ser más digital, mientras que los que utilizan facturación en papel podrían carecer de conocimiento tecnológico.

**Relación del método de pago con la cancelación del servicio:**

Los clientes que usan cheque electrónico presentan la tasa de cancelación más alta (45%), mientras que quienes utilizan cheques por correo, transferencia bancaria y tarjeta de crédito muestran tasas mucho menores y muy similares entre ellas (19%, 16% y 15%, respectivamente).
Esto sugiere que existe una mayor probabilidad de cancelación entre los clientes que usan cheque electrónico. En cambio, los métodos automáticos y el pago por correo tienden a estar más asociados con clientes que permanecen activos.

**Relación del género del cliente con la cancelación del servicio:**

La tasa de cancelación es prácticamente igual entre hombres y mujeres (26.9% y 26.1%, respectivamente). Esta mínima diferencia no sugiere ningún patrón relevante, por lo que el género del cliente no parece influir en la probabilidad de cancelación del servicio.

**Relación del estado de convivencia con la cancelación del servicio:**

La tasa de abandono es más baja entre los clientes que viven con una pareja o compañero de hogar (19%) en comparación con quienes viven solos (32%). Esto sugiere que los clientes sin compañía son más propensos a cancelar su servicio. Una posible razón es que los hogares con más de una persona suelen compartir el servicio y dividir el costo, mientras que para quienes viven solos el gasto puede resultar más significativo.

**Relación de los dependientes con la cancelación del servicio:**

La tasa de abandono es considerablemente mayor entre los clientes que no tienen dependientes (31%) en comparación con quienes sí cuentan con ellos (15%). Esto sugiere que los clientes sin dependientes tienden a cancelar con más frecuencia, posiblemente porque no tienen otros miembros del hogar que dependan del servicio, lo que reduce su nivel de compromiso con mantenerlo activo.

**Relación del tipo de servicio de internet con la cancelación del servicio:**

La tasa de abandono es considerablemente mayor entre los clientes que cuentan con fibra óptica (41%), en comparación con quienes utilizan DSL (18%) o quienes no poseen servicio de internet (7%). Esto indica una fuerte asociación entre el uso de fibra óptica y una mayor probabilidad de cancelación, por lo que los clientes con este tipo de servicio resultan claramente más propensos a darse de baja.

**Relación de la seguridad online con la cancelación del servicio:**

La tasa de abandono es considerablemente mayor entre los clientes que no cuentan con seguridad online (31%) en comparación con quienes sí la tienen (14%). Esto sugiere que los usuarios sin este servicio adicional presentan una mayor tendencia a cancelar su contrato.

**Relación de la copia de seguridad en la nube con la cancelación del servicio:**

La tasa de cancelación es ligeramente mayor entre los clientes que no cuentan con copia de seguridad en la nube (29%), en comparación con quienes sí la tienen (21%). Esto sugiere que los usuarios sin este servicio adicional presentan una tendencia ligeramente mayor a cancelar su contrato.

**Relación de la protección de dispositivo con la cancelación del servicio:**

La tasa de cancelación es ligeramente mayor entre los clientes que no cuentan con protección de dispositivo (28%) en comparación con quienes sí la tienen (22%). Esto sugiere que los usuarios sin este servicio adicional presentan una tendencia un poco mayor a cancelar su contrato.

**Relación del soporte técnico con la cancelación del servicio:**

La tasa de abandono es considerablemente mayor entre los clientes que no cuentan con soporte técnico (31%) en comparación con quienes sí lo tienen (15%). Esto sugiere que los usuarios sin este servicio adicional presentan una mayor tendencia a cancelar su contrato.

**Relación del streaming de TV con la cancelación del servicio:**

La tasa de abandono es ligeramente mayor entre los clientes que cuentan con streaming de TV (30%) en comparación con los que no lo tienen (24%). Esto sugiere que los usuarios que cuentan con este servicio presentan una ligera mayor tendencia a cancelar su contrato.

**Relación del streaming de películas con la cancelación del servicio:**

La tasa de abandono es ligeramente mayor entre los clientes que cuentan con streaming de películas (29%) en comparación con quienes no lo tienen (24%). Esto sugiere que los usuarios que tienen este servicio adicional presentan una ligera tendencia mayor a cancelar su contrato.

**Relación de la cantidad de líneas con la cancelación del servicio:**

La tasa de cancelación para los clientes que cuentan con múltiples líneas y los que no son muy similares entre sí (28% y 25%, respectivamente). Esta mínima diferencia no sugiere ningún patrón relevante, por lo que la cantidad de líneas no parece influir en la probabilidad de cancelación del servicio.

**Relación del mes de inicio del contrato con la cancelación del servicio:**

La tasa de abandono presenta variaciones importantes según el mes de inicio del contrato, con valores bajos a comienzos del año y un aumento progresivo hacia los últimos meses. Si bien el patrón es claro, los datos disponibles no permiten determinar la causa detrás de estas diferencias.

**Relación del mes de cancelación con la tasa de abandono:**

El mes de cancelación siempre presenta una tasa de abandono igual a 1, porque solo incluye casos donde el cliente abandonó. Por ello, esta variable no aporta información útil para el análisis de la cancelación.

**Relación de la edad avanzada con la cancelación del servicio:**

La tasa de abandono es considerablemente mayor entre los clientes de edad avanzada (41%) en comparación con quienes no lo son (23%). Esto indica que los adultos mayores muestran una mayor probabilidad de cancelar su contrato.

**Relación de la tenencia del servicio de internet con la cancelación del servicio:**

La tasa de cancelación es significativamente mayor entre los clientes que cuentan con servicio de internet (31%) en comparación con quienes no lo tienen (7%). Esto indica que los usuarios con servicio de internet presentan una probabilidad de cancelación mucho más alta que aquellos sin este servicio.

**Relación de la tenencia del servicio telefónico con la cancelación del servicio:**

La tasa de cancelación para los clientes que cuentan con servicio telefónico y para los que no lo tienen es bastante similar (28% y 25%, respectivamente). Esta diferencia mínima no sugiere ningún patrón relevante, por lo que el hecho de disponer o no del servicio telefónico no parece influir en la probabilidad de cancelación del contrato.

**Relación de la tenencia de un solo servicio con la cancelación del contrato:**

La tasa de abandono entre los clientes que cuentan con un solo servicio (29%) y quienes tienen ambos (25%) es muy similar. Esta diferencia mínima no revela un patrón significativo, por lo que el hecho de poseer uno o ambos servicios no parece influir de manera relevante en la probabilidad de cancelación.

### Conclusión del EDA

El análisis exploratorio revela que la cancelación del servicio está asociada principalmente con tres dimensiones: el tipo de contrato del cliente, su antigüedad, y las características de los servicios que tiene contratados.

En primer lugar, las variables relacionadas con el nivel de compromiso y la situación del hogar muestran patrones muy marcados: los clientes con contratos mensuales, que viven solos o que no tienen dependientes presentan tasas de abandono considerablemente más altas, mientras que quienes viven acompañados o tienen personas a su cargo tienden a mantener el servicio por más tiempo.

En segundo lugar, la antigüedad del cliente (tenure) es uno de los factores más determinantes. La mayoría de las cancelaciones proviene de clientes relativamente nuevos, mientras que aquellos con muchos meses de permanencia casi no desertan. Esto indica que la retención temprana es clave: si un cliente supera los primeros meses, es mucho más probable que continúe usando el servicio.

En cuanto a los servicios contratados, ciertos complementos como seguridad online, soporte técnico, copias de seguridad y protección de dispositivo están asociados con menores tasas de cancelación, lo que sugiere que los clientes que contratan servicios adicionales tienden a mantenerse activos por más tiempo. Por el contrario, los usuarios con fibra óptica y quienes utilizan métodos de pago como el cheque electrónico muestran una probabilidad significativamente mayor de cancelar.

Finalmente, algunas variables como el género, cantidad de líneas telefónicas o tener uno vs. dos servicios, no muestran relación relevante con la cancelación de contrato. El mes de cancelación tampoco aporta información útil, mientras que el mes de inicio presenta variaciones claras pero sin una causa identificable con los datos disponibles.

En conjunto, estos resultados permiten identificar perfiles de alto riesgo y factores asociados al abandono, lo cual será clave para el modelado predictivo y para el diseño de estrategias de retención.

## Modelado

### Crear nuevas variables que aporten valor al modelado a partir del EDA

In [56]:
# Creamos una variable que indique el número de servicios de protección adicionales
full_data['num_protection_addons'] = (
    full_data[['online_security','online_backup','device_protection','tech_support']]
    .apply(lambda row: (row == 'Yes').sum(), axis=1)
)

**Nota:** Durante el EDA se identificaron patrones claros asociados a clientes nuevos y a cargos altos.
Se propusieron features binarias para capturar estos efectos, pero se descartaron por redundancia con tenure y monthly_charges, ya que los modelos utilizados pueden aprender estos umbrales directamente.

In [ ]:
full_data.head()

In [ ]:
full_data.info()

### Deshacerse de las variables que conllevan fuga de datos o que no aportan nada al modelo

In [59]:
drop_features = [
    'customer_id',
    'begin_date',
    'end_date',
    'begin_month_name',
    'churn_month',
    'churn_month_name',
    'end_date_filled',
    'one_service' # esta variable se creó anteriormente porque era útil para el EDA, pero para el modelado ya no es necesaria
]

In [60]:
final_data = full_data.drop(columns=drop_features)

In [ ]:
final_data.head(30)

In [ ]:
final_data.info()

### Definir grupos de variables para el preprocesamiento

In [63]:
# Numéricas continuas
num_features = ['monthly_charges', 'total_charges', 'tenure', 'num_protection_addons']

# Binarias 0/1 (ya codificadas, se pasan tal cual al modelo)
bin_features = [
    'senior_citizen',
    'has_internet',
    'has_phone'
]

# Categóricas (incluye begin_month como categórica)
cat_features = [
    'type',
    'paperless_billing',
    'payment_method',
    'gender',
    'partner',
    'dependents',
    'internet_service',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    'multiple_lines',
    'begin_month'
]

### Definir features y target

In [64]:
features = final_data.drop(columns=['churn'])
target = final_data['churn']

### Dividir en conjuntos de entrenamiento, validación y prueba

In [65]:
features_temp, features_test, target_temp, target_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)

In [66]:
features_train, features_valid, target_train, target_valid = train_test_split(features_temp, target_temp, test_size=0.25, random_state=42, stratify=target_temp)

### Preprocesamiento (con y sin estandarización)

In [67]:
# Creamos un preprocesador sin escalado (árboles y boosting no lo necesitan)
preprocess_no_scale = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_features),
        ('bin', 'passthrough', bin_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ],
    remainder='drop'
)

# Creamos un preprocesador con StandardScaler (regresión logística y KNN lo necesitan)
preprocess_scale = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('bin', 'passthrough', bin_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ],
    remainder='drop'
)

### Grupo A: Modelos clásicos

In [ ]:
# Creamos un pipeline para cada modelo que automatiza todo el flujo de entrenamiento:
# transforma los datos, balancea las clases y entrena el modelo en un solo proceso

pipelines_A = {
    'logistic': Pipeline([
        ('preprocess', preprocess_scale),
        ('model', LogisticRegression(max_iter=2000, class_weight='balanced'))
    ]),
    'tree': Pipeline([
        ('preprocess', preprocess_no_scale),
        ('model', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
    ]),
    'rf': Pipeline([
        ('preprocess', preprocess_no_scale),
        ('model', RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced'))
    ]),
    'gb': Pipeline([
        ('preprocess', preprocess_no_scale),
        ('model', GradientBoostingClassifier(random_state=42))
    ]),
    'knn': Pipeline([
        ('preprocess', preprocess_scale),
        ('model', KNeighborsClassifier())
    ])
}

# Definimos los hiperparámetros que GridSearchCV probará mediante validación cruzada, para seleccionar la mejor versión de cada modelo
param_grids_A = {
    'logistic': {'model__C': [0.1, 1, 10]},
    'tree': {
        'model__max_depth': [None, 5, 10, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4]
    },
    'rf': {
        'model__n_estimators': [200, 500],
        'model__max_depth': [None, 10, 20]
    },
    'gb': {
        'model__n_estimators': [100, 300],
        'model__learning_rate': [0.01, 0.1]
    },
    'knn': {
        'model__n_neighbors': [3, 5, 15],
        'model__weights': ['uniform', 'distance']
    }
}

# Para cada modelo:
# 1) Buscamos los mejores hiperparámetros usando validación cruzada solo en train
# 2) Evaluamos el mejor modelo resultante en el conjunto de validación
best_models_A = {}
val_scores_A = {}

print("=== Grupo A: CV en train, evaluación en valid ===")

for name in pipelines_A:
    grid = GridSearchCV(
        estimator=pipelines_A[name],
        param_grid=param_grids_A[name],
        cv=5,
        scoring='roc_auc',
        n_jobs=-1
    )

    # Aplicamos el pipeline completo sobre el conjunto de entrenamiento, utilizando validación cruzada para seleccionar los mejores hiperparámetros
    grid.fit(features_train, target_train)
    
    # Guardamos el mejor pipeline encontrado para cada modelo
    best_models_A[name] = grid.best_estimator_
    
    # Evaluamos el modelo en el conjunto de validación
    proba_val = best_models_A[name].predict_proba(features_valid)[:, 1]
    val_scores_A[name] = roc_auc_score(target_valid, proba_val)

    print(f"{name}: CV best={grid.best_score_:.4f} | valid={val_scores_A[name]:.4f} | params={grid.best_params_}")

### Grupo B: Modelos avanzados

In [ ]:
# 1) Calculamos pesos para clases (negativos / positivos)
n_neg = (target_train == 0).sum()
n_pos = (target_train == 1).sum()

# Para XGBoost: ratio neg/pos
scale_pos_weight = n_neg / n_pos

# Para CatBoost: [peso_clase_0, peso_clase_1]
# (hacemos que la clase positiva pese más)
class_weights_cb = [1.0, scale_pos_weight]

# 2) Preprocesamos (fit solo en train)
preprocess_no_scale.fit(features_train)

features_train_proc = preprocess_no_scale.transform(features_train)
features_valid_proc = preprocess_no_scale.transform(features_valid)


# Para cada modelo:
# A diferencia del Grupo A, aquí no hacemos una búsqueda de los mejores hiperparámetros. En su lugar, monitoreamos el desempeño en validación durante el entrenamiento para controlar el sobreajuste.
# Estos modelos se entrenan con configuraciones sólidas por defecto y ajuste ligero.
val_scores_B = {}

print("\n=== Grupo B: monitoreo de desempeño y evaluación con valid ===")

# ------- Creamos XGBoost -------
xgb_model = XGBClassifier(
    n_estimators=3000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='auc',
    scale_pos_weight=scale_pos_weight
)
# Entrenamos XGBoost y utilizamos el conjunto de validación para monitorear el desempeño y aplicar early stopping
xgb_model.fit(
    features_train_proc, target_train,
    eval_set=[(features_valid_proc, target_valid)],
    early_stopping_rounds=50,
    verbose=False
)

proba_val = xgb_model.predict_proba(features_valid_proc)[:, 1]
val_scores_B['xgb'] = roc_auc_score(target_valid, proba_val)


# ------- Creamos LightGBM -------
lgbm_model = LGBMClassifier(
    n_estimators=3000,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    class_weight='balanced'
)
# Entrenamos LightGBM y utilizamos el conjunto de validación para monitorear el desempeño y aplicar early stopping
lgbm_model.fit(
    features_train_proc, target_train,
    eval_set=[(features_valid_proc, target_valid)],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(50, verbose=False)]
)

proba_val = lgbm_model.predict_proba(features_valid_proc)[:, 1]
val_scores_B['lgbm'] = roc_auc_score(target_valid, proba_val)


# ------- Creamos CatBoost -------
cat_model = CatBoostClassifier(
    iterations=3000,
    learning_rate=0.05,
    depth=6,
    random_seed=42,
    verbose=0,
    eval_metric='AUC',
    class_weights=class_weights_cb,
    od_type='Iter',
    od_wait=50
)
# Entrenamos CatBoost y utilizamos el conjunto de validación para monitorear el desempeño y aplicar early stopping
cat_model.fit(
    features_train_proc, target_train,
    eval_set=(features_valid_proc, target_valid),
    use_best_model=True
)

proba_val = cat_model.predict_proba(features_valid_proc)[:, 1]
val_scores_B['catboost'] = roc_auc_score(target_valid, proba_val)

# Mostramos las puntuaciones
print(val_scores_B)

### Ranking de modelos

In [ ]:
# Ordenamos los modelos de mejor a peor según su AUC-ROC en validación
final_val_scores = {**val_scores_A, **val_scores_B}
ranking = sorted(final_val_scores.items(), key=lambda x: x[1], reverse=True)

# Mostramos los modelos y su puntuación
print("\n=== Ranking de modelos según AUC-ROC en validación ===")
for name, score in ranking:
    print(name, round(score, 4))

### Evaluación final del mejor modelo en el conjunto de prueba

In [ ]:
# 1. Elegimos el mejor modelo según el ranking en validación
best_model_name, best_val_score = ranking[0]
print(f"\nMejor modelo según validación: {best_model_name} (AUC={best_val_score:.4f})")

# 2. Entrenamos el mejor modelo en train+valid
features_train_full = pd.concat([features_train, features_valid])
target_train_full = pd.concat([target_train, target_valid])


if best_model_name in best_models_A:
    # -------------------------------
    # Modelos del grupo A (pipelines)
    # -------------------------------
    best_model = best_models_A[best_model_name]

    # Reentrenamos el pipeline completo desde cero con train+valid
    best_model.fit(features_train_full, target_train_full)

    # Predicción en test
    proba_test = best_model.predict_proba(features_test)[:, 1]

else:
    # -------------------------------
    # Modelos del grupo B (boosting)
    # -------------------------------
    # Preprocesamiento
    preprocess_no_scale.fit(features_train_full)
    features_train_full_proc = preprocess_no_scale.transform(features_train_full)
    features_test_proc = preprocess_no_scale.transform(features_test)

    # Re-creamos el modelo según el nombre
    if best_model_name == 'xgb':
        final_model = XGBClassifier(
            n_estimators=3000,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            eval_metric='auc',
            scale_pos_weight=scale_pos_weight
        )
    elif best_model_name == 'lgbm':
        final_model = LGBMClassifier(
            n_estimators=3000,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            class_weight='balanced'
        )
    elif best_model_name == 'catboost':
        final_model = CatBoostClassifier(
            iterations=3000,
            learning_rate=0.05,
            depth=6,
            random_seed=42,
            verbose=0,
            eval_metric='AUC',
            class_weights=class_weights_cb
        )

    # Entrenamos el modelo final
    final_model.fit(features_train_full_proc, target_train_full)

    # Predicción en test
    proba_test = final_model.predict_proba(features_test_proc)[:, 1]


# 3. Calculamos AUC-ROC en test
test_auc = roc_auc_score(target_test, proba_test)

print(f"\nAUC-ROC final en TEST: {test_auc:.4f}")

**Estrategia de evaluación de modelos**

Para la selección del modelo predictivo se implementó una estrategia de evaluación en dos niveles. En primer lugar, para los modelos de menor costo computacional (Logistic Regression, KNN, Decision Tree, Random Forest y Gradient Boosting de scikit-learn), se utilizó validación cruzada mediante GridSearchCV con el objetivo de optimizar hiperparámetros, aplicándose únicamente sobre el conjunto de entrenamiento. Este proceso permitió obtener, para cada modelo, una configuración optimizada sin utilizar información del conjunto de validación.

Posteriormente, los modelos ya optimizados se evaluaron sobre un conjunto de validación independiente para compararlos entre sí utilizando la métrica ROC-AUC. Para modelos más complejos como XGBoost, LightGBM y CatBoost, no se empleó validación cruzada exhaustiva debido a su mayor costo computacional; en su lugar, se usaron configuraciones iniciales bien establecidas y early stopping para controlar el sobreajuste. Finalmente, el modelo seleccionado se evaluó una sola vez sobre el conjunto de prueba, garantizando una comparación justa y evitando fuga de información.

### Conclusiones

En este proyecto se desarrolló un modelo predictivo para estimar la probabilidad de cancelación de clientes del operador de telecomunicaciones Interconnect, utilizando información contractual, personal y de servicios contratados. El problema se abordó como una tarea de clasificación binaria, donde el objetivo fue identificar si un cliente cancela o no su contrato.

A partir del análisis exploratorio de datos, se identificaron patrones claros asociados a la cancelación del servicio. Los clientes con contratos mensuales, menor antigüedad, cargos mensuales elevados y menos servicios adicionales presentan una probabilidad significativamente mayor de cancelar. Asimismo, variables relacionadas con el contexto del hogar, como no vivir con una pareja o no tener dependientes, también se asociaron con mayores tasas de abandono. Por el contrario, clientes con mayor antigüedad y con servicios adicionales como soporte técnico o seguridad online muestran una mayor permanencia.

Para el modelado predictivo se evaluaron distintos algoritmos de clasificación, incluyendo modelos lineales, modelos basados en árboles y técnicas de boosting. La selección del modelo se realizó utilizando la métrica ROC-AUC sobre un conjunto de validación independiente. El modelo con mejor desempeño fue CatBoost, que alcanzó un AUC-ROC de 0.939 en el conjunto de prueba, lo que indica una alta capacidad para diferenciar entre clientes que cancelan y clientes que permanecen activos.

Este resultado indica que el modelo puede utilizarse como una herramienta de apoyo para identificar clientes con mayor probabilidad de cancelar su contrato. En particular, permite detectar clientes en riesgo con anticipación, lo que podría ayudar a la empresa a tomar acciones de retención a tiempo.

Desde el punto de vista del negocio, el modelo podría servir para priorizar clientes a los que se les puedan ofrecer promociones, descuentos o servicios adicionales, especialmente en el caso de clientes nuevos o con contratos mensuales.

Como limitación, el modelo se basa únicamente en la información disponible en los datos proporcionados y no considera otros factores que podrían influir en la cancelación. Además, no se define un umbral específico para la toma de decisiones, por lo que este debería ajustarse según los objetivos del negocio.

En conclusión, el proyecto demuestra que es posible predecir la cancelación de clientes utilizando los datos disponibles y que estos resultados pueden servir como apoyo para futuras estrategias de retención en Interconnect.